In [38]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
from sklearn.model_selection import cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import Lasso

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import os
import xgboost as xgb

# Function to calculate correlation after imputation
def evaluate_imputation(data, column, target, strategy, constant_value=None):
    """
    Evaluate the correlation between a column and the target column after imputation.

    Parameters:
        data (pd.DataFrame): The dataset containing the column and target.
        column (str): The name of the column to impute.
        target (str): The name of the target column.
        strategy (str): The imputation strategy ('mean', 'constant').
        constant_value: The value to use for constant imputation (only needed if strategy is 'constant').

    Returns:
        float: Correlation between the imputed column and the target column.
    """
    imputer = None
    if strategy == 'constant':
        if constant_value is None:
            raise ValueError("Constant value must be provided for 'constant' strategy.")
        imputer = SimpleImputer(strategy='constant', fill_value=constant_value)
    elif strategy == 'mean':
        imputer = SimpleImputer(strategy='mean')
    else:
        raise ValueError(f"Unsupported strategy: {strategy}")

    # Impute the missing values
    imputed_column = imputer.fit_transform(data[[column]])
    
    # Replace original column with imputed values for correlation calculation
    data_imputed = data.copy()
    data_imputed[column] = imputed_column
    
    # Calculate correlation
    correlation = data_imputed[column].corr(data_imputed[target])
    return correlation
    
def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

def clean_data(df):
    """
    Cleans the dataset by handling missing values, infinite values, and preparing it for analysis.
    
    Parameters:
    - df: pd.DataFrame - The dataset to clean.
    
    Returns:
    - cleaned_df: pd.DataFrame - The cleaned dataset.
    """


    print("\nCleaning Data...")
    # Replace infinities with NaN
    df.replace([float('inf'), -float('inf')], float('nan'), inplace=True)
    
    # Report missing values before cleaning
    missing_before = df.isnull().sum().sum()
    print(f"Missing values before cleaning: {missing_before}")
    
    # Fill NaN values with column means for numerical features
    df.fillna(df.mean(numeric_only=True), inplace=True)
    
    # Report missing values after cleaning
    missing_after = df.isnull().sum().sum()
    print(f"Missing values after cleaning: {missing_after}")
    
    return df
    
def analyze_data(df, target_column=None, correlation_columns=None, plot_columns=None, calculate_correlations=False, calculate_mi=False, correlations_threshold = 0.9):
    """
    Analyze data by calculating statistics, correlations, unique values, empty values, and creating visualizations.
    
    Parameters:
    - df: pd.DataFrame - The dataset to analyze.
    - target_column: str - The target column for correlation analysis.
    - correlation_columns: list - List of columns to calculate correlations with the target column.
    - plot_columns: list - List of columns to visualize trends or distributions.
    
    Returns:
    - stats: pd.DataFrame - Summary statistics of the dataset.
    - correlations: pd.Series or None - Correlations with the target column, if specified.
    - unique_values: pd.Series - Count of unique values per column.
    - empty_values: pd.Series - Count of empty (NaN) values per column.
    """

    print("\nShape:")
    print(df.shape)
    
    print("\nInfo:")
    df.info()
    
    pd.set_option('display.max_columns', None)
    df_encoded = pd.get_dummies(df, drop_first=True, dtype=int)

    # 1. Display descriptive statistics
    print("\nDescriptive Statistics:")
    stats = df.describe(include='all')
    #with pd.option_context('display.max_rows', None):  # Show all rows
    #    print(stats)
    
    # 2. Check unique values
    print("\nUnique Object Values per Column > 100:")
    unique_values = df.nunique()
    #with pd.option_context('display.max_rows', None):  # Show all rows
    #    print(unique_values)

    result = {col: df[col].nunique() for col in df.select_dtypes(include='object') if df[col].nunique() > 100}

    print(result)
    
    # 3. Check empty (NaN) values
    print("\nEmpty (NaN) Values per Column > 50%:")
    empty_values = df.isnull().sum()
    #with pd.option_context('display.max_rows', None):  # Show all rows
    #    print(empty_values)
    threshold = 0.5
    empty_columns = df.columns[df.isnull().mean() > threshold]
    
    # Display the result
    print(empty_columns.tolist())

    # 4. Calculate correlations
    correlations = None
    if calculate_correlations and target_column:
        if correlation_columns is None:
            correlation_columns = [col for col in df.columns if col != target_column]
        print(f"\nCorrelations with '{target_column}':")
        correlations = df[correlation_columns].corrwith(df[target_column]).abs().sort_values(ascending=False)
        print(correlations)

    # 4. Calculate simple correlations
    corr_matrix = df_encoded.corr()
    print("\High correlations")
    # Highlight correlations above a threshold
    high_corr = corr_matrix[(corr_matrix > threshold) & (corr_matrix != 1.0)]

    print("\nHighly Correlated Pairs (Threshold > "+ str(correlations_threshold) +"):")

    if correlations_threshold >0:
        for col in high_corr.columns:
            for row in high_corr.index:
                if pd.notnull(high_corr.loc[row, col]):
                    print(f"{row} and {col} have correlation: {corr_matrix.loc[row, col]}")

    # 5. Visualize data
    if plot_columns:
        print("\nVisualizing data:")
        for column in plot_columns:
            if pd.api.types.is_numeric_dtype(df[column]):
                # Histogram for numerical data
                sns.histplot(df[column], kde=True)
                plt.title(f"Distribution of {column}")
                plt.show()
            else:
                # Bar plot for categorical data
                sns.countplot(data=df, x=column)
                plt.title(f"Countplot of {column}")
                plt.show()
    
    # 6. Calculate MI scores
    mi_scores = None
    if calculate_mi and target_column:
        print("\nMutual Information (MI) Scores:")
        X = df_encoded.drop(columns=[target_column])
        y = df_encoded[target_column]
        mi_scores_array = mutual_info_regression(X, y, random_state=42)
        mi_scores = pd.Series(mi_scores_array, index=X.columns).sort_values(ascending=False)
        
        with pd.option_context('display.max_rows', None):  # Show all rows
            print(mi_scores)
     
    return stats, correlations, unique_values, empty_values

Load and check the data 

In [39]:
# Any results you write to the current directory are saved as output.

train_data = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
train_data.head()
test_data = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")
test_data.head()

#mapping numeric ordinal
mapping = {
    'Ex': 5,
    'Gd': 4,
    'TA': 3,
    'Fa': 2,
    'Po': 1,
    'NA': 0  # or NaN if you prefer
}
#ExterQual
#ExterCond
#BsmtQual
#BsmtCond
#HeatingQC
#KitchenQual
#FireplaceQu
#GarageQual
#GarageCond
#PoolQC

train_data["BsmtBath"] = train_data["BsmtFullBath"] + train_data["BsmtHalfBath"]
train_data["AboveGradeBath"] = train_data["FullBath"] + train_data["HalfBath"]
train_data["TotalPorchArea"] = (train_data["WoodDeckSF"] + 
                                 train_data["OpenPorchSF"] + 
                                 train_data["EnclosedPorch"] + 
                                 train_data["3SsnPorch"] + 
                                 train_data["ScreenPorch"])
train_data['ExterQual'] = train_data['ExterQual'].map(mapping)
train_data['ExterCond'] = train_data['ExterCond'].map(mapping)
train_data['BsmtQual'] = train_data['BsmtQual'].map(mapping)
train_data['BsmtCond'] = train_data['BsmtCond'].map(mapping)
train_data['HeatingQC'] = train_data['HeatingQC'].map(mapping)
train_data['KitchenQual'] = train_data['KitchenQual'].map(mapping)
train_data['FireplaceQu'] = train_data['FireplaceQu'].map(mapping)
train_data['GarageQual'] = train_data['GarageQual'].map(mapping)
train_data['GarageCond'] = train_data['GarageCond'].map(mapping)
train_data['PoolQC'] = train_data['PoolQC'].map(mapping)

test_data["BsmtBath"] = test_data["BsmtFullBath"] + test_data["BsmtHalfBath"]
test_data["AboveGradeBath"] = test_data["FullBath"] + test_data["HalfBath"]
test_data["TotalPorchArea"] = (test_data["WoodDeckSF"] + 
                                 test_data["OpenPorchSF"] + 
                                 test_data["EnclosedPorch"] + 
                                 test_data["3SsnPorch"] + 
                                 test_data["ScreenPorch"])
test_data['ExterQual'] = test_data['ExterQual'].map(mapping)
test_data['ExterCond'] = test_data['ExterCond'].map(mapping)
test_data['BsmtQual'] = test_data['BsmtQual'].map(mapping)
test_data['BsmtCond'] = test_data['BsmtCond'].map(mapping)
test_data['HeatingQC'] = test_data['HeatingQC'].map(mapping)
test_data['KitchenQual'] = test_data['KitchenQual'].map(mapping)
test_data['FireplaceQu'] = test_data['FireplaceQu'].map(mapping)
test_data['GarageQual'] = test_data['GarageQual'].map(mapping)
test_data['GarageCond'] = test_data['GarageCond'].map(mapping)
test_data['PoolQC'] = test_data['PoolQC'].map(mapping)

print("\nStart DATA\n")
print(train_data.head())

df = pd.DataFrame(train_data)

stats, correlations, unique_values, empty_values = analyze_data(
    df,
    target_column="SalePrice",  # Example target column
    calculate_mi = False
)

# Check the correlation between OverallQual and OverallCond
print("\ncorrelation between OverallQual and OverallCond")
print(train_data[["OverallQual", "OverallCond"]].corr())

# Check the correlation between OverallQual and OverallCond
print("\ncorrelation between YearBuilt and YearRemodAdd")
print(train_data[["YearBuilt", "YearRemodAdd"]].corr())

# Check the correlation between GarageCars and GarageArea
print("\ncorrelation between GarageCars and GarageArea")
print(train_data[["GarageCars", "GarageArea"]].corr())


Start DATA

   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities LotConfig LandSlope Neighborhood Condition1  \
0         Lvl    AllPub    Inside       Gtl      CollgCr       Norm   
1         Lvl    AllPub       FR2       Gtl      Veenker      Feedr   
2         Lvl    AllPub    Inside       Gtl      CollgCr       Norm   
3         Lvl    AllPub    Corner       Gtl      Crawfor       Norm   
4         Lvl    AllPub       FR2       Gtl      NoRidge       Norm   

  Condition2 BldgType HouseStyle  OverallQual  OverallCond  YearBuilt  \
0       Norm     1Fam     2Story          

In [40]:
td = pd.concat([train_data, test_data], ignore_index=True, sort  = False)

#column with low correlation
columns_to_drop = ["Id","MoSold", "YrSold", "MiscFeature", "RoofMatl", "LowQualFinSF", "Condition2", "LandSlope","Street"]

#column with high correlation, and we feel it's not correlated with price
columns_to_drop.extend(["SaleType", "SaleCondition", "Exterior2nd"])

#BsmtFinSF1	BsmtFinSF2 BsmtUnfSF =	TotalBsmtSF
columns_to_drop.extend(["BsmtFinSF1", "BsmtFinSF2", "BsmtUnfSF"])

##1stFlrSF + 2ndFlrSF = GrLivArea
columns_to_drop.extend(["1stFlrSF", "2ndFlrSF"])

#optimize batthroom
##BsmtFullBath: Basement full bathrooms
#BsmtHalfBath: Basement half bathrooms
#FullBath: Full bathrooms above grade
#HalfBath: Half baths above grade
#replaced with BsmtBath and AboveGradeBath
columns_to_drop.extend(["BsmtFullBath", "BsmtHalfBath", "FullBath", "HalfBath"])

#GarageCars and GarageArea, correlated, so we just remove GarageCars
columns_to_drop.extend(["GarageCars"])

#we combine the following to TotalPorchArea
columns_to_drop.extend(["WoodDeckSF","OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch"])

#Dropped columns due to high missing values:
columns_to_drop.extend(["Alley","MasVnrType","PoolQC","Fence","MiscFeature"])

td = td.drop(columns=columns_to_drop)

#OneHotEncoder
#td = pd.get_dummies(td, drop_first = True)

print("\nFINAL DATA\n")
print(td.head())
#td still have SalePrice

X_test = td[td.SalePrice.isnull()]


FINAL DATA

   MSSubClass MSZoning  LotFrontage  LotArea LotShape LandContour Utilities  \
0          60       RL         65.0     8450      Reg         Lvl    AllPub   
1          20       RL         80.0     9600      Reg         Lvl    AllPub   
2          60       RL         68.0    11250      IR1         Lvl    AllPub   
3          70       RL         60.0     9550      IR1         Lvl    AllPub   
4          60       RL         84.0    14260      IR1         Lvl    AllPub   

  LotConfig Neighborhood Condition1 BldgType HouseStyle  OverallQual  \
0    Inside      CollgCr       Norm     1Fam     2Story            7   
1       FR2      Veenker      Feedr     1Fam     1Story            6   
2    Inside      CollgCr       Norm     1Fam     2Story            7   
3    Corner      Crawfor       Norm     1Fam     2Story            7   
4       FR2      NoRidge       Norm     1Fam     2Story            8   

   OverallCond  YearBuilt  YearRemodAdd RoofStyle Exterior1st  MasVnrArea  \
0 

1. Drop not important column
2. update columns
   1. Preprocessing for numerical data, if empty, then numberic column change to median value
   2. for string, if it's empty then set to most frequently use
   3. Apply one hot 

proprocessor manually

In [41]:
X = td[td.SalePrice.notnull()]
y = X[[ 'SalePrice']].copy()
y_aligned = y.iloc[1:]  # Removes the first row

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print("Original Data")
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X.columns if
                    X[cname].nunique() < 10 and 
                    X[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X.columns if 
                X[cname].dtype in ['int64', 'float64']]

special_na_zero_cols = ["MasVnrArea"]

#remove special_na_zero_cols from numerical_cols
numerical_cols = [col for col in numerical_cols if col not in special_na_zero_cols]

# Preprocessing for numerical data, if empty, then numberic column change to median value, median should be better
numerical_transformer = SimpleImputer(strategy='median')
numeric_na_zero_transformer = SimpleImputer(strategy='constant', fill_value=0)  # For the special column

# Preprocessing for categorical data
#imputer change null to numeric
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('special', numeric_na_zero_transformer, special_na_zero_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Apply preprocessing manually
preprocessor.fit(X_train)  # Fit the preprocessor on training data
X_train_preprocessed = preprocessor.transform(X_train)  # Transform training data
X_val_preprocessed = preprocessor.transform(X_val)      # Transform validation data
X_test_preprocessed = preprocessor.transform(X_test)

print("After processing Data")
print(X_train_preprocessed.shape)
print(X_val_preprocessed.shape)
print(X_test_preprocessed.shape)
print(X_test_preprocessed)
print(y.shape)
print(y.head())

# Convert transformed data to DataFrame if needed (optional)
# This step is useful if you want feature names for debugging.
X_train_preprocessed = pd.DataFrame(X_train_preprocessed)
X_val_preprocessed = pd.DataFrame(X_val_preprocessed)

####################################
# check correlation after preprocessed
####################################
# If y_train is a Series or array, make sure it's a DataFrame or Series for concatenation
if isinstance(y_train, pd.Series):
    y_train = y_train.to_frame()  # Convert Series to DataFrame

# Combine X_train and y_train into a single DataFrame
combined_train_df = pd.concat([X_train.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1)
print(combined_train_df)
stats, correlations, unique_values, empty_values = analyze_data(
    combined_train_df,
    target_column="SalePrice",  # Example target column
    calculate_mi = True,
    correlations_threshold = 0,
    calculate_correlations =True
)


Original Data
(1168, 53)
(292, 53)
(1168, 1)
(292, 1)
After processing Data
(1168, 132)
(292, 132)
(1459, 132)
[[2.0000e+01 8.0000e+01 1.1622e+04 ... 0.0000e+00 0.0000e+00 1.0000e+00]
 [2.0000e+01 8.1000e+01 1.4267e+04 ... 0.0000e+00 0.0000e+00 1.0000e+00]
 [6.0000e+01 7.4000e+01 1.3830e+04 ... 0.0000e+00 0.0000e+00 1.0000e+00]
 ...
 [2.0000e+01 1.6000e+02 2.0000e+04 ... 0.0000e+00 0.0000e+00 1.0000e+00]
 [8.5000e+01 6.2000e+01 1.0441e+04 ... 0.0000e+00 0.0000e+00 1.0000e+00]
 [6.0000e+01 7.4000e+01 9.6270e+03 ... 0.0000e+00 0.0000e+00 1.0000e+00]]
(1460, 1)
   SalePrice
0   208500.0
1   181500.0
2   223500.0
3   140000.0
4   250000.0
      MSSubClass MSZoning  LotFrontage  LotArea LotShape LandContour  \
0             20       RL         70.0     8400      Reg         Lvl   
1             60       RL         59.0     7837      IR1         Lvl   
2             30       RL         67.0     8777      Reg         Lvl   
3             50       RL         60.0     7200      Reg         Lvl 

ValueError: Input X contains NaN.

In [ ]:
#, early_stopping_rounds=50
model = xgb.XGBRegressor(n_estimators=20000, learning_rate=0.1, max_depth=6,enable_categorical=True)
# Define the simplified pipeline with only the model
my_pipeline = Pipeline(steps=[
    ('model', model)
])

for col in categorical_cols:
    X_train[col] = X_train[col].astype('category')
    X_val[col] = X_val[col].astype('category')

my_pipeline.fit(
    X_train_preprocessed, y_train,
    model__eval_set=[(X_val_preprocessed, y_val)],  # Pass preprocessed validation data
    model__early_stopping_rounds=50,               # Set early stopping rounds
    model__verbose=True                            # Enable verbose output for training progress
)
##############################
#let XGBRegressor handle mising value and one hot
###############################
#my_pipeline.fit(
#    X_train, y_train,
#    model__eval_set=[(X_val, y_val)],  # Pass preprocessed validation data
#    model__early_stopping_rounds=50,               # Set early stopping rounds
#    model__verbose=True                            # Enable verbose output for training progress
#)


# Parameter grid for GridSearchCV
param_grid = {
    'model__n_estimators': [20000],
    'model__max_depth': [6,8,10],
    'model__learning_rate': [0.1]#0.01,0.05,0.1,0.2],
}

# GridSearchCV
#grid_search = GridSearchCV(my_pipeline, param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2)

# Fit the pipeline with GridSearchCV
#grid_search.fit(X_train_preprocessed, y_train)

# Best parameters and score
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_}")
##Best parameters: {'model__learning_rate': 0.1, 'model__max_depth': 6, 'model__n_estimators': 20000}
#Best score: -35096476.62818256

# Preprocessing of validation data, get predictions
predictions_xgb = my_pipeline.predict(X_test_preprocessed)

score = mean_squared_error(y_aligned, predictions_xgb)
print('MSE:', score)


In [ ]:
# Train LASSO regression
lasso = Lasso(alpha=0.1)  # Alpha is the regularization strength
lasso.fit(X_train_preprocessed, y_train)

# Evaluate
y_pred = lasso.predict(X_test_preprocessed)
mse = mean_squared_error(y_aligned, y_pred)
print("lasso Mean Squared Error:", mse)

In [ ]:
output = pd.DataFrame({'Id': test_data.Id, 'SalePrice': predictions_xgb})
output.SalePrice = output.SalePrice.astype(int)
print(output.shape)

output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

print(output.head())

X_train_preprocessed.columns = preprocessor.get_feature_names_out()
X_train_preprocessed.to_csv('transformed_data_train.csv', index=False)